# 9. Многотабличные запросы

# 9.1 Объединение с помощью UNION

Объедините с помощью UNION таблицы bank_transactions и cashbox_transactions.

**Решение:**

```MySQL
SELECT DATE_FORMAT(date, '%d.%m.%y') AS date, amount, 'bank' AS payment_type
FROM bank_transactions
WHERE client_id = 56
UNION
SELECT DATE_FORMAT(date, '%d.%m.%y') AS date, amount, 'cash' AS payment_type
FROM cashbox_transactions
WHERE client_id = 56;
```

После слияния двух компаний, IT отделу поручили объединение баз данных. Слейте users и members в единую таблицу со следующими столбцами:

1. id — идентификатор пользователя. Так как идентификаторы пользователей в таблицах одинаковые, то примените к ним следующую формулу id * 10 + N, где N – равен одному для таблицы users и двум для members.
2. first_name — имя пользователя. Используйте оригинальное значение из 3. таблицы users и извлеките имя из поля name таблицы members.
3. last_name — фамилия пользователя. Используйте оригинальное значение из таблицы users и извлеките фамилию из поля name таблицы members.
4. age — возраст пользователя. Используйте оригинальное значение из таблицы users и NULL для данных из members.
5. birthday — день рождения пользователя. Используйте оригинальное значение из таблицы members и NULL для данных из users.
6. sex — пол пользователя.
**Решение:**

```MySQL
SELECT id * 10 + 1 AS id,
       first_name,
       last_name,
       age,
       NULL AS birthday,
       sex
FROM users

UNION

SELECT id * 10 + 2 AS id,
       SUBSTRING_INDEX(name, ' ', 1) AS first_name,
       SUBSTRING_INDEX(name, ' ', -1) as last_name,
       NULL AS age,
       birthday,
       sex
FROM members;
```

Каждый регион ведет базу автомобилей и их номеров в своей собственной уникальной базе.
Ваша задача — объединить таблицы из разных регионов так, чтобы получилась таблица со следующий структурой:

1. number — номер автомобиля в формате а111аа.
2. region — регион регистрации автомобиля (целое число).
3. mark — марка автомобиля.
4. model — модель автомобиля.

**Решение:**

```MySQL
SELECT LEFT(number, 6) AS number,
       CAST(RIGHT(number, 2) AS UNSIGNED) AS region,
       mark,
       model
FROM cars

UNION

SELECT number,
       39 AS region,
       mark,
       model
FROM region39

UNION

SELECT LOWER(number) AS number,
       region,
       mark,
       model
FROM avto

UNION

SELECT LEFT(number, 6) AS number,
       CAST(RIGHT(number, 2) AS UNSIGNED) AS region,
       SUBSTRING_INDEX(car, ' ', 1) AS mark,
       SUBSTRING_INDEX(car, ' ', -1)AS model
FROM autos;
```

# 9.2 Объединение с помощью UNION: сортировка

В целях оптимизации производительности в базе данных есть две таблицы для хранения объявлений: advs и closed_advs, в первой хранятся активные объявления, а во второй неактивные.

Основаная часть запросов приходится на advs, но когда пользователь заходит в личный кабинет, то видит все свои объявления из обеих таблиц.

Объедините обе таблицы с помощью UNION и получите данные для пользователя с идентификатором 45. Выведите колонки id, category_id, date, text, а также status (статус объявления). Для открытых объявлений в поле status укажите «opened» а для закрытых — «closed».

Итоговую таблицу отсортируйте по дате, а затем по id.

**Решение:**

```MySQL
(SELECT id, category_id, date, text, 'opened' AS status
FROM advs
WHERE user_id = 45)
 
UNION

(SELECT id, category_id, date, text, 'closed' AS status
FROM closed_advs
WHERE user_id = 45)

ORDER BY date, id;
```

Объедините с помощью UNION данные из таблиц bank_transactions, cashbox_transactions и paypal_transactions. Выведите три столбца:

1. date — дату и время транзакции;
2. amount — сумму транзакции;
2. pt — вид платежа: «bank» для банковских транзакций и «cash» для наличности и «paypal» для платежей в системе PayPal.

Итоговые данные отсортируйте по дате в обратном хронологическом порядке.

**Решение:**

```MySQL
(SELECT date, amount, 'bank' AS pt 
 FROM bank_transactions)
 
UNION

(SELECT date, amount, 'cash' AS pt
 FROM cashbox_transactions)
 
UNION

(SELECT date, amount, 'paypal' AS pt 
 FROM paypal_transactions)
 
ORDER BY date DESC;
```

На главной странице сайта выводится ТОП10 компьютерных игр. Чтобы вывод был разнообразным — берут по две самых популярных игры из 5 категорий, а затем сортируют полученные данные по рейтингу в обратном порядке. Строки с одинаковым рейтингом отсортируйте по id.

Получите из таблицы games данные для вывода на главную страницу сайта для категорий: 1 - Action, 2 - RPG, 3 - Adventure, 4 - Strategy и 5 - Shooter. Выведите поля id, name, rating и genre, где genre — название категории.

**Решение:**

```MySQL
(select  id, name, rating, 'Action' as genre
 from games
 where category_id = 1
 order by rating desc limit 2)
 
union

(select  id, name, rating, 'RPG' as genre
 from games
 where category_id = 2
 order by rating desc limit 2)
 
union

(select  id, name, rating, 'Adventure' as genre
 from games
 where category_id = 3
 order by rating desc limit 2)
 
union

(select  id, name, rating, 'Strategy' as genre
 from games
 where category_id = 4
 order by rating desc limit 2)
 
union

(select  id, name, rating, 'Shooter' as genre
 from games
 where category_id = 5
 order by rating desc limit 2)
 
order by rating desc, id;
```

# 9.3 Объединение с помощью UNION: группировка

Платежи клиентов в базе данных хранятся в трех таблицах:
    1. bank_transactions — прямые платежи на расчетный счет,
    2. cashbox_transactions — платежи наличными в кассу организации,
    3. paypal_transactions — платежи через сервис PayPal.

Получите виды платежей и суммы по каждому виду. Виды платежей выведите в колонке type: «bank» для банковских транзакций и «cash» для наличности и «paypal» для платежей в системе PayPal. Суммы платежей в колонке sum_amount.

Итоговые данные отсортируйте по сумме.

**Решение:**

```MySQL
(SELECT 'bank' AS type, SUM(amount) AS sum_amount 
 FROM bank_transactions)
 
UNION

(SELECT 'cash' AS type, SUM(amount) AS sum_amount
 FROM cashbox_transactions)
 
UNION

(SELECT 'paypal' AS type, SUM(amount) AS sum_amount
 FROM paypal_transactions)
 
ORDER BY sum_amount;
```

Платежи клиентов в базе данных хранятся в трех таблицах:
    1. bank_transactions — прямые платежи на расчетный счет,
    2. cashbox_transactions — платежи наличными в кассу организации,
    3. paypal_transactions — платежи через сервис PayPal.

Получите суммы входящих платежей по всем таблицам с разбивкой по годам и месяцам. Итоговая таблица должна содержать 3 столбца: year, month, month_amount. Итоговые данные отсортируйте по году и месяцу.

**Решение:**

```MySQL
SELECT year, month, SUM(amount) as month_amount

FROM

(SELECT YEAR(date) AS year, MONTH(date) AS month, amount
 FROM bank_transactions
UNION
SELECT YEAR(date) AS year, MONTH(date) AS month, amount
 FROM cashbox_transactions
UNION
SELECT YEAR(date) AS year, MONTH(date) AS month, amount
 FROM paypal_transactions) 
 
 transactions
 
GROUP BY year, month
ORDER BY year, month;
```

Платежи клиентов в базе данных хранятся в трех таблицах:
    1. bank_transactions — прямые платежи на расчетный счет,
    2. cashbox_transactions — платежи наличными в кассу организации,
    3. paypal_transactions — платежи через сервис PayPal.

Получите общую сумму всех платежей клиентов. Колонку с результатом назовите all_money.

**Решение:**

```MySQL
SELECT SUM(all_money) AS all_money

FROM

(SELECT SUM(amount) AS all_money
 FROM bank_transactions
UNION
SELECT SUM(amount) AS all_money
 FROM cashbox_transactions
UNION
SELECT SUM(amount) AS all_money
 FROM paypal_transactions) 
 
 transactions;
```

# 9.4 Отношение один к одному

В таблице users хранится базовая информация о пользователях, а в таблице users_details — подробная. Таблицы связаны отношением один-к-одному так, что первичные ключи в таблицах совпадают.

Получите id пользователя, его имя, фамилию и биографию в одном запросе.

**Решение:**

```MySQL
SELECT u.id, u.first_name, u.last_name, ud.bio
FROM users AS u, users_details AS ud
WHERE u.id = ud.id
```

В базе данных интернет-магазина по продаже красок есть две таблицы: products и products_details.

В первой хранится общая информация о краске: название, количество, цена, категория.
А в таблице products_details содержатся дополнительные текстовые данные: область применения, описание, схема распыления.

Таблицы связаны отношением один-к-одному, но не через первичные ключи, а альтернативным способом. Первичный ключ id в таблице products связан с колонкой product_id в таблице products_details.

Получите в одном запросе id, название товара, а также всю информацию из таблицы products_details кроме id и product_id.
Данные отсортируйте по цене.

**Решение:**

```MySQL
SELECT products.id, products.name, products_details.description
FROM products, products_details
WHERE products.id = products_details.product_id
ORDER BY products.price
```

Регистрационные данные пользователей хранятся в таблице users, а персональные данные в users_p. Таблицы связаны друг с другом отношением один-к-одному так, что первичные ключи совпадают.

Получите идентификатор, имя и фамилию для пользователей зарегистрировавшихся (date_joined) после полуночи 1 января 2016 года с паспортами серия которых начинается на 32.

Отсортируйте данные по фамилии пользователей.

**Решение:**

```MySQL
SELECT u.id, u.first_name, u.last_name
FROM users AS u, users_p AS up
WHERE u.id = up.id AND u.date_joined > '2016-01-01 00:00:00' AND up.series LIKE '32%'
ORDER BY u.last_name;
```

В таблице users хранится базовая информация о пользователях, а в таблице users_details — подробная. Таблицы связаны отношением один-к-одному так, что первичные ключи в таблицах совпадают.

Обновите данные о пользователе c id равным 8: измените email на «karina.n@domain.com», а фамилию на «Некифорова». Выполните все изменения в одном SQL запросе.

**Решение:**

```MySQL
UPDATE users AS u, users_details AS ud
SET u.email = 'karina.n@domain.com', ud.last_name = 'Некифорова'
WHERE u.id = ud.id AND u.id = 8;
```

В базе данных интернет-магазина по продаже красок есть две таблицы: products и products_details.

В первой хранится общая информация о краске: название, количество, цена, категория.
А в таблице products_details содержатся дополнительные текстовые данные: область применения, описание, схема распыления.

Таблицы связаны отношением один-к-одному. Первичный ключ id в таблице products связан с колонкой product_id в таблице products_details.

Очистите поле description для всех неактивных товаров. Используйте один SQL-запрос.

**Решение:**

```MySQL
UPDATE products, products_details
SET products_details.description = ''
WHERE products.id= products_details.product_id AND products.active = FALSE;
```

В базе данных интернет-магазина по продаже красок есть две таблицы: products и products_details.

В первой хранится общая информация о краске: название, количество, цена, категория.
А в таблице products_details содержатся дополнительные текстовые данные: область применения, описание, схема распыления.

Таблицы связаны отношением один-к-одному. Первичный ключ id в таблице products связан с колонкой product_id в таблице products_details, а также для него указано ON DELETE CASCADE.

Удалите из обеих таблиц все товары, которых нет в наличии или которые неактивны.
Используйте один SQL-запрос.

**Решение:**

```MySQL
DELETE p, pd
FROM products AS p, products_details AS pd
WHERE p.id = pd.product_id 
AND (p.active IS FALSE OR  p.count = 0)
```

Иногда нужно удалить данные из одной таблицы, основываясь на условиях другой. Для этого в конструкции DELETE предусмотрен специальный оператор USING.

Разберем на примере ниже, как он работает.
1. В первой строке мы пишем из какой таблицы будем удалять — table2.
2. Затем с помощью USING и JOIN указываем, что эта таблица связана с таблицей table1.
3. Далее идет блок WHERE, где мы сперва задаем связь между двумя таблицами — через их id.
4. И в конце мы указываем условие удаления table1.id < 10.

Получается, что мы удаляем данные из таблицы table2, но по условию в таблице table1.

**Решение:**

```MySQL
DELETE FROM users_details
    USING users JOIN users_details
    WHERE 
    users_details.id = users.id
    AND users.active = 0 ;
```

```MySQL
DELETE FROM ud
USING users_details as ud 
JOIN users as u 
    WHERE 
    ud.id = u.id AND 
    u.active IS FALSE
```

# 9.5 Внешний ключ

В базе данных есть таблица users, которая была создана с помощью следующего запроса:

```MySQL
CREATE TABLE users (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    date_joined DATETIME NOT NULL
)
```
Создайте таблицу users_data, в которой первичный ключ id будет также внешним ключом с ссылкой на таблицу users. Помимо первичного ключа, таблица должна содержать поле bio типа TEXT.

**Решение:**

```MySQL
CREATE TABLE users_data (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    bio TEXT,
    FOREIGN KEY (id) REFERENCES users(id)
);
```

В базе данных интернет-магазина по продаже красок есть таблица products, которая была создана с помощью следующего запроса:

```MySQL
CREATE TABLE products (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50) NULL,
    count INTEGER NULL,
    price INTEGER NULL
);
```

Создайте таблицу products_details, которая будет содержать дополнительные текстовые данные о краске и связана с таблицей products отношением один-к-одному.

В таблице должно быть три поля: id, product_id и description.
Поле product_id должно быть внешним ключом, который ссылается на поле id в таблице products.

(обратите внимание, что внешний ключ должен быть точно такого же типа, что и поле, на которое он ссылается)
Поле description должно быть типа TEXT.

**Решение:**

```MySQL
CREATE TABLE products_details (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    product_id INT UNSIGNED NOT NULL UNIQUE KEY,
    description TEXT,
    FOREIGN KEY (product_id) REFERENCES products(id)
);
```

В таблице users хранится базовая информация о пользователях, а в таблице users_details — подробная. Таблицы связаны отношением один-к-одному так, что первичные ключи в таблицах совпадают, но могут быть рассинхронизированы (id в одной таблице могут опережать ключи в другой). 

Добавьте нового пользователя со следующими данными:
   Имя: Антон
   Фамилия: Дмитриев
   Биография: «Антон родился в 1993 году.»

**Решение:**

```MySQL
INSERT INTO users (first_name, last_name)
VALUES ('Антон','Дмитриев');

INSERT INTO users_details (id, bio)
VALUES (LAST_INSERT_ID(),'Антон родился в 1993 году.');
```

В таблице users хранится базовая информация о пользователях, а в таблице users_details — подробная. Таблицы связаны отношением один-к-одному так, что первичные ключи в таблицах совпадают.

Добавьте Антону Дмитриеву следующую биографию: «Антон родился в 1993 году.»
А Василисе Кац измените биографию на «Василиса Кац родилась в 1995 году.»

**Решение:**

```MySQL
INSERT INTO users_details (id, bio)
VALUES (15,'Антон родился в 1993 году.');

UPDATE users_details
SET bio = 'Василиса Кац родилась в 1995 году.'
WHERE id = 4;
```

# 9.6 Отношения один-ко-многим

В таблице products находятся товары, а в categories — категории, к которым они относятся. Поле category_id является внешним ключом, который указывает на первичный ключ категории.

Получите в одном запросе имя товара, цену товара и имя категории, к которой принадлежит товар.
Отсортируйте данные в алфавитном порядке по имени товара.
Имя категории выведите в колонке с именем category.

**Решение:**

```MySQL
SELECT p.name, p.price, c.name AS category
FROM products AS p, categories AS c
WHERE p.category_id = c.id
ORDER BY name;
```

В таблице users содержатся данные сотрудников, а в deals сделки этих сотрудников. Таблицы связаны через внешний ключи user_id, который находится в deals и указывает на столбец id в таблице users.

Получите фамилии, имена и суммы сделок всех сотрудников. Вывод отсортируйте по сумме в обратном порядке, а сами значения введите в колонке total.

**Решение:**

```MySQL
SELECT u.last_name, u.first_name, SUM(d.amount) as total
FROM users AS u, deals AS d
WHERE d.user_id = u.id
GROUP BY u.id
ORDER BY SUM(d.amount) DESC;
```

Таблицы orders и users связанны отношением многие к одному.
orders содержит внешний ключ user_id, который ссылается на id пользователя.

Получите из таблиц дату и стоимость каждого выполненного заказа, а также фамилию и имя пользователя, который этот заказ оформил.
Информацию отсортируйте по дате заказа, дату выведите в формате ДД.ММ.ГГГГ в поле date

**Решение:**

```MySQL
SELECT DATE_FORMAT(o.date, '%d.%m.%Y') as date , o.amount, u.last_name, u.first_name
FROM orders AS o, users AS u
WHERE
o.user_id = u.id AND o.status = 'completed'
ORDER BY o.date;
```

В таблице products хранятся товары, а в categories — категории, к которым они относятся. Поле category_id является внешним ключом, который указывает на первичный ключ категории.

Получите список категорий с количеством наименований товаров в каждой.
Данные отсортируйте в алфавитном порядке по имени категории.
Имя категории выведите в поле category, а количество товаров в products.

**Решение:**

```MySQL
SELECT categories.name AS category, count(products.name ) AS products
FROM products, categories
WHERE products.category_id = categories.id
GROUP BY categories.name
ORDER BY category;
```

В таблице users хранятся данные по всем пользователям сайта поликлиники: клиентам и докторам. Доктора отмечены в поле is_doctor.

В таблице calendar содержатся данные о времени записи (visit_date) пациентов (client_id) к докторам (doctor_id). И doctor_id и client_id ссылаются на поле id в таблице users.

Составьте расписание для доктора Анастасии Дейчман (id равен 9) на 17 апреля 2017 года.

Выведите фамилию пациента, имя пациента и время приема в формате ЧЧ:ММ.
Время приема выведите в колонке visit_time.
Данные должны быть отсортированы в хронологическом порядке.

**Решение:**

```MySQL
SELECT u.last_name,u.first_name, DATE_FORMAT(c.visit_date,'%H:%i') AS visit_time
FROM users as u,calendar as c
WHERE
u.id=c.client_id
AND c.doctor_id=9
AND DATE(c.visit_date)='2017-04-17'
ORDER BY c.visit_date;
```

Таблицы orders и users связанны отношением многие к одному.
orders содержит внешний ключ user_id, который ссылается на id пользователя.

Получите все заказы, которые сделали женщины от 18 лет и старше в феврале 2017 года.
Выводить нужно только столбцы таблицы orders.

Данные отсортируйте по стоимости заказа.

**Решение:**

```MySQL
SELECT o.*
FROM users as u, orders as o
WHERE o.user_id = u.id
AND u.sex = 'w'
AND u.age >= 18
AND MONTH(o.date) = 2
AND YEAR(o.date) = 2017
ORDER BY o.amount;
```

В таблице users хранится информация о сотрудниках компании, а в arrival содержатся данные о дате (a_date) и времени (a_time) прихода сотрудников на работу.

Сотрудники, которые пришли на работу после 9 часов утра, считаются опоздавшими. Выведите id, фамилию и имя сотрудников, которые опоздали хотя бы раз в марте 2017 года.

**Решение:**

```MySQL
SELECT DISTINCT u.id, u.last_name, u.first_name
FROM users as u, arrival as a
WHERE u.id = a.user_id
AND MONTH(a.a_date) = 3
AND a.a_time > '09:00:00';
```

Один из популярных приемов организации иерархии — это создание внешнего ключа, который ссылается на ту же таблицу, в которой он находится.

Например, в таблице categories есть поле parent_id, которое указывает на id. Если посмотреть на записи с id равным 11 (Кофе) и 13 (Чай), то увидим, что parent_id для них равен 10 — Кофе и чай.
Сама запись с id равным 10, в parent_id содержит NULL, то есть не имеет родительской категории или, другими словами, находится на верхнем уровне иерархии.

Получите из таблицы categories все категории, у которых родительской является «Напитки». Выведите только id и название категорий. Данные отсортируйте в алфавитном порядке.

**Решение:**

```MySQL
SELECT id, name
FROM categories
WHERE parent_id = 5
ORDER BY name ASC;
```

В базе данных есть три таблицы: products, categories и brands.

Выведите id и название товара, название категории и название бренда для товаров, которые есть на складе.

Название товара выведите в поле name, категорию в category, бренда в brand.
Отсортируйте данные по цене, а затем по id товара.

**Решение:**

```MySQL
SELECT products.id, products.name , 
categories.name AS category, 
brands.name AS brand

FROM products, categories, brands
WHERE brands.id=products.brand
AND categories.id= products.category
AND products.count > 0
ORDER BY products.price, products.id;
```

В базе данных есть три таблицы: products, categories и brands.

Получите из таблицы products все джинсы и юбки компании Mango.
Выведите только id, название и цену. Данные отсортируйте по цене, а затем по id. Не выводите товары, которых нет в наличии.

**Решение:**

```MySQL
SELECT p.id, p.name, p.price
FROM products AS p, categories AS c, brands AS b
WHERE p.brand = b.id
AND p.category = c.id
AND (c.name = 'Джинсы' OR c.name = 'Юбки')
AND count > 0
AND brand = 1
ORDER BY p.price, p.id;
```

Получите все товары из категории «Напитки» и всех её подкатегорий.
Выведите id, имя и стоимость товара, а также название категории.
Название категории выведите в колонке category.

Данные отсортируйте по названию категории, а затем по имени товара.

**Решение:**

```MySQL
SELECT products.id,products.name, products.price, categories.name AS category
FROM categories, products
WHERE products.category_id = categories.id
AND (categories.parent_id = 5 OR products.category_id = 5)
ORDER BY categories.name, products.name;
```

Пользователь книжного интернет-магазина решил найти все книги по MySQL. Помогите ему.

В каждой строке итоговой таблицы выведите id, название и стоимость книги, а также автора: имя и фамилию в одном поле с названием author.

Данные отсортируйте по названию книг.

**Решение:**

```MySQL
SELECT b.id, b.name, b.price, CONCAT(a.first_name, ' ', a.last_name) as author FROM  authors AS a, books AS b 
WHERE  a.id = b.author_id AND b.name LIKE '%MySQL%'
ORDER BY b.name;
```

# 9.7 Создание связей один-ко-многим

В базе данных есть две таблицы: genres и artists. В первой находится список жанров, а во второй — исполнители. Сейчас таблицы никак не связаны между собой.
Создайте в таблице artists новое поле genre_id и сделайте его внешним ключом на поле id в таблице genres.

Исходные запросы для создания таблиц были такие:

```MySQL
CREATE TABLE genres (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50) NULL
);
CREATE TABLE artists (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50) NULL,
    is_group BOOLEAN NULL
);
```
Учтите, что обе таблицы уже заполнены данными.

**Решение:**

```MySQL
ALTER TABLE artists
ADD COLUMN genre_id INT UNSIGNED BULL;

ALTER TABLE artists
ADD FOREIGN KEY (genre_id) REFERENCES genres(id)
ON DELETE SET NULL;
```

В базе данных есть две таблицы: products и categories. В первой находится список товаров, а во второй — категории. Сейчас таблицы связаны между собой через поле category_id, однако это поле не является настоящим внешним ключом.
Сделайте category_id внешним ключом на поле id в таблице categories.

Исходные запросы для создания таблиц были такие:

```MySQL
CREATE TABLE products (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50) NULL,
    count INTEGER NULL,
    price INTEGER NULL,
    category_id INTEGER UNSIGNED NOT NULL
);
CREATE TABLE categories (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50) NULL
);
```
Учтите, что обе таблицы уже заполнены данными.

**Решение:**

```MySQL
ALTER TABLE products
ADD FOREIGN KEY (category_id) REFERENCES categories (id);
```

В таблице users хранятся данные по всем пользователям сайта поликлиники: клиентам и докторам. Доктора отмечены в поле is_doctor. Создайте таблицу calendar для хранения записей пациентов к врачам. Таблица должна иметь 4 поля:

id — идентификатор записи.
doctor_id — внешний ключ на колонку id в таблице users, NULL запрещен.
client_id — также внешний ключ на колонку id в таблице users, NULL запрещен.
visit_date — дата и время визита, NULL запрещен.

Таблица users была создана с помощью запроса:

```MySQL
CREATE TABLE users (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    first_name VARCHAR(50) NULL,
    last_name VARCHAR(50) NULL,
    is_doctor BOOLEAN NULL
);
```
Учтите, что обе таблицы уже заполнены данными.

**Решение:**

```MySQL
CREATE TABLE calendar (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    doctor_id INT UNSIGNED NOT NULL,
    client_id INT UNSIGNED NOT NULL,
    visit_date DATETIME NOT NULL,
FOREIGN KEY (doctor_id) REFERENCES users(id),
FOREIGN KEY (client_id) REFERENCES users(id)
    );
```

В таблице users хранятся данные по всем пользователям сайта поликлиники: клиентам и докторам. Доктора отмечены в поле is_doctor.
В таблице calendar содержатся данные о времени записи (visit_date) пациентов (client_id) к докторам (doctor_id). И doctor_id и client_id ссылаются на поле id в таблице users.

Запишите Светлану Иванову к массажисту Лиане Белой.
Добавьте 5 ежедневных сеансов начиная с 20 мая 2017 года.
Каждый сеанс начинается в 12:30.

**Решение:**

```MySQL
INSERT INTO calendar (doctor_id, client_id, visit_date)
VALUES 
(7, 2, '2017-05-20 12:30:00'),
(7, 2, '2017-05-21 12:30:00'),
(7, 2, '2017-05-22 12:30:00'),
(7, 2, '2017-05-23 12:30:00'),
(7, 2, '2017-05-24 12:30:00');
```

# 9.8 JOIN

Перепишите SQL-запрос с помощью конструкции JOIN:

```MySQL
SELECT 
    p.id, 
    p.name, 
    p.price,
    cats.name as category
FROM 
    products as p, 
    categories as cats
WHERE
    p.category_id = cats.id AND
    cats.parent_id IS NULL
```

**Решение:**

```MySQL
SELECT p.id, p.name, p.price, cats.name AS category
FROM products AS p
JOIN categories AS cats ON p.category_id = cats.id
WHERE cats.parent_id IS NULL;
```

В таблице artists представлены исполнители, а в таблице genres — жанры в которых они играют.

Получите названия всех групп и жанры, в которых они играют.
Жанры выведите в поле genre
Отсортируйте данные по имени жанра, а затем по названию группы.

**Решение:**

```MySQL
SELECT a.name, g.name as genre
FROM genres as g
JOIN artists as a ON g.id = a.genre_id
WHERE is_group = TRUE
ORDER BY genre, a.name;
```

Получите id и названия жанров, а также количество исполнителей в них.
Количество исполнителей выводите в поле artists, а названия жанров в name.
Жанры, в которых нет исполнителей, выводить не надо.

Данные отсортируйте сперва по количеству исполнителей в обратном порядке, а потом по названию жанра.

**Решение:**

```MySQL
SELECT genres.id, genres.name AS name, COUNT(artists.name) AS artists
FROM genres
JOIN artists ON genres.id=artists.genre_id
GROUP BY genres.id
HAVING COUNT(artists.name) > 0
ORDER BY COUNT(artists.name) DESC, genres.name;
```

Получите список городов-миллионников в Европе.

Выведите название города, название страны и население города.
Название страны выведите в поле country, города в поле name, а население в population.
Данные отсортируйте по населению в обратном порядке (крупные города выше).

Поле pw в таблице countries представлено типом SET.

**Решение:**

```MySQL
SELECT ci.name as name, co.name as country, ci.population as population
FROM countries as co
JOIN cities as ci ON ci.country = co.id
WHERE ci.population >= 1000000 AND FIND_IN_SET('Europe', co.pw)
ORDER BY ci.population DESC;
```

На сайте интернет-магазина нужно выводить только те категории, в которых есть товары.

Получите имена категорий, в которых есть товары, а также количество товаров в каждой категории.

Данные выведите в двух колонках:
    1. category с именем категории;
    2. products с количеством товаров.
Отсортируйте итоговую таблицу по имени категории.

**Решение:**

```MySQL
SELECT c.name as category, SUM(p.count) as products
FROM categories as c
JOIN products as p ON p.category = c.id
GROUP BY c.name
HAVING products > 0
ORDER BY category;
```

Добавьте в таблицы новый жанр «Rap» и исполнителя «Eminem» в этом жанре.

**Решение:**

```MySQL
INSERT INTO genres (id, name)
VALUES
(6, 'Rap');

INSERT INTO artists (id, name, genre_id,    is_group)
VALUES
(11, 'Eminem', 6, False);
```

# 9.9 Понимание JOIN

В таблице roles хранится список ролей (должностей) в компании, а в employees — сотрудники с указанием их роли через поле role_id. Некоторые сотрудники не имеют ролей.

Получите список всех активных сотрудников с указанием их должностей.
Если у сотрудника нет должности, то нужно вывести NULL.
Выведите 3 столбца: имя, фамилию и должность (role).
Данные отсортируйте по фамилии, а после по имени в алфавитном порядке.

**Решение:**

```MySQL
SELECT employees.first_name, employees.last_name, roles.name AS role
FROM employees
LEFT JOIN roles
ON roles.id = employees.role_id
WHERE active = True
ORDER BY employees.last_name, employees.first_name;
```

В таблице artists представлены исполнители, а в таблице genres — жанры в которых они играют.

Получите названия всех жанров и количество исполнителей в каждом жанре.
Жанры выведите в поле genres, а количество исполнителей в artists.
Отсортируйте данные по названию жанра.

**Решение:**

```MySQL
SELECT g.name AS genres,COUNT(a.genre_id) AS artists
FROM genres AS g 
LEFT JOIN artists as a 
ON g.id = a.genre_id
GROUP BY g.id
ORDER BY g.name;
```

В таблице artists представлены исполнители, а в таблице genres — жанры в которых они играют.

Получите названия всех жанров и количество исполнителей в каждом жанре.
Жанры выведите в поле genres, а количество исполнителей в artists.
Отсортируйте данные по названию жанра.

**Решение:**

```MySQL
SELECT c.name AS category, IFNULL(SUM(p.count), 0) AS products
FROM categories AS c
LEFT JOIN products AS p ON c.id = p.category
GROUP BY c.id
ORDER BY category;
```

В таблице products находятся товары, а в categories — категории, к которым они относятся. Поле category_id является внешним ключом, который указывает на первичный ключ категории.

Получите в одном запросе имя товара, цену товара и имя категории, к которой принадлежит товар.

Отсортируйте данные в алфавитном порядке по имени товара.
Имя категории выведите в колонке с именем category.
Если у товара нет категории, то выведите NULL.

**Решение:**

```MySQL
SELECT products.name, products.price, categories.name AS category
FROM products
LEFT JOIN categories ON products.category_id=categories.id
ORDER BY products.name;
```

В таблице roles хранится список ролей (должностей) в компании, а в employees — сотрудники с указанием их роли через поле role_id.

Получите список всех должностей и количество активных сотрудников для каждой должности.

Выведите 2 столбца: название должности (name) и количество сотрудников (employees).

Данные сперва отсортируйте по числу сотрудников в обратном порядке, а затем по имени роли.

**Решение:**

```MySQL
SELECT roles.name AS name, COUNT(employees.active) AS employees
FROM roles
LEFT JOIN employees ON employees.role_id=roles.id
WHERE employees.active IS true OR employees.active IS NULL 
GROUP BY roles.id
ORDER BY employees DESC, roles.name ASC;
```

# 9.10 Понимание JOIN, часть 2

Получите все данные о книгах и писателях. Свяжите книги с их авторами если это возможно.
Выведите 5 столбцов:

author_id — id автора, NULL для книг, у которых автор не указан;
book_id — id книги, которую написал автор, NULL — если у автора нет книг;
last_name — фамилия автора;
first_name — имя автора;
name — название книги.

Данные отсортируйте сперва по id авторов, а затем по id книг.

**Решение:**

```MySQL
SELECT a.id AS author_id, b.id AS book_id, a.last_name AS last_name, a.first_name AS first_name, b.name AS name
FROM books AS b
LEFT JOIN authors AS a ON b.author_id = a.id

UNION

SELECT a.id AS author_id, b.id AS book_id, a.last_name AS last_name, a.first_name AS first_name, b.name AS name
FROM books AS b
RIGHT JOIN authors AS a ON b.author_id = a.id

ORDER BY author_id, book_id;
```

Получите все страны и города из таблиц countries и cities.
Свяжите города со странами, в которых они находятся.

Выведите 2 столбца:
    1. country — название страны;
    2. city — название города.

Данные отсортируйте сперва по названию страны, а затем по названию города.

**Решение:**

```MySQL
SELECT c.name AS country,ci.name as city 
FROM countries AS c
LEFT  JOIN cities AS ci ON c.id = ci.country

UNION

SELECT c.name AS country,ci.name as city 
FROM countries AS c
RIGHT  JOIN cities AS ci ON ci.country  = c.id

ORDER BY country, city;
```

# 9.11 Выборка из трех и более таблиц

Получите список всех пользователей (users) с указанием их ролей (roles) и отделов (departments).

Выведите 4 поля: last_name, first_name, role (название роли) и department (название отдела).

Данные отсортируйте сперва по фамилии, а затем по имени.

**Решение:**

```MySQL
SELECT u.last_name AS last_name, u. first_name AS first_name, r.name AS role, d.name AS department
FROM users AS u
LEFT JOIN departments AS d ON u.department_id=d.id
LEFT JOIN roles AS r ON u.role_id=r.id
ORDER BY u.last_name, u. first_name;
```

База данных автосалона содержит 3 таблицы: marks, models и cars.
Таблица models связана с marks через поле mark_id, а таблица cars с models через поле model_id. То есть cars напрямую не связана с marks, однако эту связь можно проследить через models.

Получите все автомобили салона с указанием модели и марки. Выведите следующие поля: id автомобиля, марку автомобиля (поле mark), модель автомобиля (поле model) и его цену. Данные отсортируйте по цене в обратном порядке.

**Решение:**

```MySQL
SELECT c.id, m.name as mark, mm.name as model, c.price
FROM marks as m
JOIN models as mm ON m.id = mm.mark_id
JOIN cars as c ON mm.id = c.model_id
ORDER BY c.price DESC;  
```

База данных автосалона содержит 3 таблицы: marks, models и cars.

Получите все марки представленные в салоне, а также суммарную стоимость автомобилей каждой марки. Выведите 2 колонки:

1. mark — название марки;
2. sum — стоимость всех автомобилей этой марки.
Данные отсортируйте по названию марки.

**Решение:**

```MySQL
SELECT
    marks.name as mark,
    SUM(cars.price)as sum
FROM marks
LEFT JOIN models ON marks.id = models.mark_id
JOIN cars ON models.id = cars.model_id
GROUP BY marks.name
ORDER BY mark;
```

База данных музыкального сайта состоит из 4 таблиц: genres, artists, albums и songs.

Получите все песни в жанрах Rock и Metal с 2008 по 2010 год включительно.
Выведите id песни, её название, а также название альбома в поле album и название исполнителя в поле artist. Данные отсортируйте по названию композиции.

**Решение:**

```MySQL
SELECT genres.name  as name,count(songs.id) as songs
FROM artists
JOIN genres ON genres.id = artists.genre_id
JOIN albums ON artists.id = albums.artist_id
JOIN songs ON albums.id = songs.album_id
group by name
order by name;
```

В CRM системе компании есть 4 таблицы:

1. managers — менеджеры (сотрудники) компании;
2. companies — контрагенты компании;
3. clients — сотрудники контрагентов;
4. calls — звонки сотрудников контрагентов менеджерам компании. В таблице также фиксируется дата (date) и продолжительность (duration_sec) звонка в секундах.

Получите данные о звонках за 5 апреля 2018 года.
Выведите 5 колонок:

time — время звонка в формате ЧЧ:ММ;
manager — имя и фамилия менеджера, который принял звонок;
client — имя и фамилия сотрудника контрагента, который позвонил;
company — название компании контрагента, NULL если названия нет;
duration — продолжительность звонка в формате Ч:ММ:СС.
Данные отсортируйте по дате и времени звонка.

**Решение:**

```MySQL
SELECT date_format(ca.date,'%H:%i') AS time,
CONCAT(m.first_name,' ', m.last_name) AS manager,
CONCAT(cl.first_name,' ', cl.last_name)AS client,
co.name AS company, sec_to_time(ca.duration_sec) AS duration
FROM managers AS m
LEFT JOIN calls AS ca ON m.id = ca.manager_id
LEFT JOIN clients AS cl ON cl.id = ca.client_id
LEFT JOIN companies AS co ON co.id = cl.company_id
WHERE DATE(ca.date) = '2018-04-05'
ORDER BY time, duration;
```

В CRM системе компании есть 4 таблицы:

1. managers — менеджеры (сотрудники) компании;
2. companies — контрагенты компании;
3. clients — сотрудники контрагентов;
4. calls — звонки сотрудников контрагентов менеджерам компании. В таблице также фиксируется дата (date) и продолжительность (duration_sec) звонка в секундах.

Получите список всех компаний и общую продолжительность звонков по каждой из них.
Выведите 2 колонки:

1. company — название компании;
2. duration — общая продолжительность звонков в формате Ч:ММ:СС.

Данные отсортируйте по продолжительности разговоров.

**Решение:**

```MySQL
SELECT co.name as company, SEC_TO_TIME(IFNULL(SUM(duration_sec),0)) as duration 
FROM companies as co 
LEFT JOIN clients as cl ON cl.company_id = co.id 
LEFT JOIN calls as c ON c.client_id = cl.id 
GROUP BY co.name 
ORDER BY duration;
```

В CRM системе компании есть 4 таблицы:

1. managers — менеджеры (сотрудники) компании;
2. companies — контрагенты компании;
3. clients — сотрудники контрагентов;
4. calls — звонки сотрудников контрагентов менеджерам компании. В таблице также фиксируется дата (date) и продолжительность (duration_sec) звонка в секундах.

Получите среднее время общения менеджеров с сотрудниками Cloud Computing.

Выведите 3 колонки:

1. first_name — имя менеджера;
2. last_name — фамилия менеджера;
3. avg_duration — средняя продолжительность звонка в формате ЧЧ:ММ:СС.

Данные отсортируйте по средней продолжительности разговоров в обратном порядке.

**Решение:**

```MySQL
SELECT managers.first_name as first_name,managers.last_name as last_name, DATE_FORMAT(SEC_TO_TIME(AVG(calls.duration_sec)),'%H:%i:%S') as avg_duration
FROM managers
RIGHT JOIN calls ON managers.id = calls.manager_id
RIGHT JOIN clients ON clients.id =calls.client_id
RIGHT JOIN companies ON companies.id = clients.company_id
WHERE companies.name = 'Cloud Computing'
GROUP BY managers.id
ORDER BY avg_duration DESC;
```

# 9.12 Ссылочная целостность

В таблице products находятся товары, а в categories — категории, к которым они относятся. Поле category_id является внешним ключом, который указывает на первичный ключ категории. При создании ключа в запросе указали ON DELETE RESTRICT.

Удалите категории «Молочные продукты» и «Мясо» вместе с товарами.

**Решение:**

```MySQL
DELETE FROM products
WHERE category_id = 2 OR category_id = 17;

DELETE FROM categories
WHERE id = 2 OR id = 17;
```

В таблице products находятся товары, а в categories — категории, к которым они относятся. Поле category_id является внешним ключом, который указывает на первичный ключ категории. При создании ключа в запросе указали ON DELETE SET NULL.

Удалите категории «Молочные продукты» и «Мясо» без удаления товаров.

**Решение:**

```MySQL
DELETE FROM categories
WHERE id = 2 OR id = 17;
```

В таблице products находятся товары, а в categories — категории, к которым они относятся. Поле category_id является внешним ключом, который указывает на первичный ключ категории. При создании ключа в запросе указали ON DELETE CASCADE.

Удалите категории «Молочные продукты» и «Мясо» вместе с товарами.

**Решение:**

```MySQL
DELETE FROM categories
WHERE id = 2 OR id = 17;
```

В таблице products находятся товары, а в categories — категории, к которым они относятся. Поле category_id является внешним ключом, который указывает на первичный ключ категории. При создании ключа в запросе указали ON DELETE SET NULL.

Перенесите товары из категории «Фрукты и овощи» в категории «Фрукты» и «Овощи» а затем удалите «Фрукты и овощи»

Также удалите категории «Молочные продукты» вместе с товарами.

**Решение:**

```MySQL
UPDATE products
SET category_id = 18
WHERE id = 3
  OR id = 4
  OR id = 9;
 
UPDATE products
SET category_id = 19
WHERE id = 19;

DELETE FROM products
WHERE category_id = 2;

DELETE FROM categories
WHERE name = 'Фрукты и овощи';

DELETE FROM categories
WHERE name = 'Молочные продукты';
```

# 9.13 Отношения многие ко многим

Таблицы users и roles связаны отношением многие ко многим через таблицу users_roles.

Получите всех программистов компании.

Выведите id пользователя, его имя и фамилию. Данные отсортируйте по фамилии.

**Решение:**

```MySQL
SELECT users_roles.user_id as id,
       users.first_name, users.last_name
FROM users
LEFT JOIN users_roles ON users.id = users_roles.user_id
WHERE users_roles.role_id = 3
ORDER BY users.last_name;
```

Таблицы users и roles связаны отношением многие ко многим через таблицу users_roles.

Получите список всех ролей и количество сотрудников в каждой роли.

Выведите два поля: role — название должности и members — количество сотрудников в этой должности. Данные отсортируйте по названию должности.

**Решение:**

```MySQL
SELECT roles.name as role, COUNT(users.id) as members 
FROM roles
JOIN users_roles ON users_roles.role_id = roles.id
JOIN users ON users.id = users_roles.user_id
GROUP BY role
ORDER BY role;
```

Таблицы users и roles связаны отношением многие ко многим через таблицу users_roles.

Получите список всех сотрудников, которые выполняют более одной роли.
Выведите id, имена и фамилии таких сотрудников. Данные отсортируйте по id.

**Решение:**

```MySQL
SELECT u.id, first_name, last_name 
FROM users as u
JOIN users_roles as ur on u.id=ur.user_id
JOIN roles as r on r.id=ur.role_id
GROUP BY u.id
HAVING COUNT(u.id)>1;
```

Таблицы users и roles связаны отношением многие ко многим через таблицу users_roles.

Получите список всех сотрудников, которые не выполняют ни одной роли.
Выведите id, имена и фамилии таких сотрудников. Данные отсортируйте по id.

**Решение:**

```MySQL
SELECT users.id,users.first_name,users.last_name
FROM users_roles
RIGHT JOIN users ON users.id=users_roles.user_id
LEFT JOIN roles ON roles.id=users_roles.role_id
GROUP BY users.id
HAVING count(roles.id)=0;
```

Таблицы users и roles связаны отношением многие ко многим через таблицу users_roles.

В компании произошли небольшие изменения и вам необходимо внести соответствующие правки в таблицы:
    1. Уберите Светлану Иванову из менеджеров.
    2. Сделайте менеджером Анастасию Дейчман.
    3. Назначьте Александра Дмитриева программистом.

**Решение:**

```MySQL
DELETE FROM users_roles
      WHERE user_id = 2 AND role_id = 1;
 
INSERT INTO users_roles (user_id, role_id)
     VALUES (9, 1),
            (10, 3);
```

Таблицы users, roles и projects связаны отношением многие ко многим через таблицу users_rp.

Такая «тройная» связь позволяет задавать роли пользователей в конкретных проектах.

Получите состав команды с указанием ролей для проекта «Сайт оконный»
Выведите id, имена и фамилии сотрудников, а также их роли в поле role.
Данные отсортируйте по фамилии.

**Решение:**

```MySQL
SELECT u.id, u.first_name, u.last_name, r.name AS role
FROM roles AS r
JOIN users_rp AS ur ON r.id = ur.role_id
JOIN users AS u ON ur.user_id = u.id
JOIN projects AS p ON ur.project_id = p.id
WHERE p.name = 'Сайт оконный'
ORDER BY u.last_name;
```

Рассмотрим базу данных интернет-магазина:

1. Таблица orders содержит список заказов пользователей и связана с users отношением один ко многим через поле user_id.

2. Также orders связана отношением многие ко многим с таблицей products через таблицу orders_details, что дает возможность указывать какие именно товары находятся в заказе. Одна запись в orders_details соответствует одному заказанному товару.

3. Поле count в таблице products отвечает за количество товаров на складе.

Получите выполненные заказы с указанием количества и стоимости товаров в них.
Выведите три поля: id заказа, количество товаров в заказе (в поле products) и сумму заказа (в поле amount).

Данные отсортируйте по сумме заказа.

**Решение:**

```MySQL
SELECT o.id, COUNT(od.product_id) as products, SUM(p.price) AS amount
FROM orders as o
JOIN orders_details as od 
ON o.id = od.order_id
JOIN products as p ON p.id = od.product_id
WHERE o.status = 'success'
GROUP BY o.id
ORDER BY amount;
```

Продолжаем работать с базой данных интернет-магазина:

1. Таблица orders содержит список заказов пользователей и связана с users отношением один ко многим через поле user_id.

2. Также orders связана отношением многие ко многим с таблицей products через таблицу orders_details, что дает возможность указывать какие именно товары находятся в заказе.

При проектировании программист ошибся и сделал возможность удаления товаров из таблицы products с каскадным удалением из таблицы orders_details. И забыл. Через год в магазине каких-то товаров не стало и менеджер удалил их из таблицы, потянув за собой все данные о заказе, кроме самого заказа. В итоге у нас есть выполненные заказы без товаров.

Получите выполненные заказы, в которых нет товаров.
Выведите три поля: id заказа, статус заказа, а также количество товаров в заказе (в поле products).

Данные отсортируйте по id заказа.

**Решение:**

```MySQL
SELECT o.id,o.status,COUNT(od.order_id) as products
FROM orders as o
LEFT JOIN users as u ON u.id=o.user_id
LEFT JOIN orders_details as od ON o.id = od.order_id
LEFT JOIN products as p ON p.id = od.product_id
WHERE o.status = 'success'
GROUP BY o.id
HAVING products = 0;
```

В базе данных музыкального сайта 5 таблиц: genres — для жанров, artists — для исполнителей, artists_genres — таблица для связи исполнителей с жанрами, albums — для альбомов (связанная с исполнителями через поле artist_id) и songs — для хранения композиций (связать с альбомами через поле album_id).

Получите все композиции исполнителей в жанрах Rock и Metal начиная с 2008 года.
Выведите id и название композиции, альбом (в поле album), исполнителя (в поле artist), а также год альбома (в поле year).

Данные отсортируйте по году выхода, а затем по id композиции.

**Решение:**

```MySQL
SELECT s.id, s.name, al.name AS album, ar.name AS artist, al. year AS year
FROM songs AS s
LEFT JOIN albums AS al ON al.id=s.album_id
LEFT JOIN artists AS ar ON ar.id= al.artist_id
LEFT JOIN artists_genres AS ag ON ag.artist_id = ar.id
LEFT JOIN genres AS g ON g.id=ag.genre_id
WHERE (g.name ='Rock' OR g.name ='Metal') AND al.year >=2008
GROUP BY s.id
ORDER BY al.year, s.id;
```

Продолжаем работать с базой данных интернет-магазина:

1. Таблица orders содержит список заказов пользователей и связана с users отношением один ко многим через поле user_id.

2. Также orders связана отношением многие ко многим с таблицей products через таблицу orders_details, что дает возможность указывать какие именно товары находятся в заказе.

Получите 5 самых продаваемых товаров (тех, которых больше всего покупали). Учитывайте только выполненные заказы.

Выведите четыре поля: id и название товара, количество проданных товаров (в поле sold), общую стоимость проданных товаров (в поле total).

Данные отсортируйте сперва по полю sold в обратном порядке, а затем по total, также в обратном порядке.

**Решение:**

```MySQL
SELECT products.id, products.name, COUNT(products.count) AS sold, SUM(products.price) AS total
FROM products
JOIN orders_details ON orders_details.product_id = products.id
JOIN orders ON orders.id = orders_details.order_id
WHERE orders.status = 'success'
GROUP BY products.id
ORDER BY sold DESC, total DESC
LIMIT 5;
```

Продолжаем работать с базой данных интернет-магазина:

1. Таблица orders содержит список заказов пользователей и связана с users отношением один ко многим через поле user_id.

2. Также orders связана отношением многие ко многим с таблицей products через таблицу orders_details, что дает возможность указывать какие именно товары находятся в заказе.

Заказы со статусом «new» считаются корзиной. Получите состав корзины Александра Дмитриева.

Выведите три поля: id, название и стоимость товара.
Данные отсортируйте по id товара.

**Решение:**

```MySQL
SELECT products.id,name,price
FROM products
LEFT JOIN orders_details ON orders_details.product_id=products.id
LEFT JOIN orders ON orders.id=orders_details.order_id
LEFT JOIN users ON users.id=orders.user_id
WHERE orders.status='new' AND users.id=10
ORDER BY id;
```

Продолжаем работать с базой данных интернет-магазина:

1. Таблица orders содержит список заказов пользователей и связана с users отношением один ко многим через поле user_id.

2. Также orders связана отношением многие ко многим с таблицей products через таблицу orders_details, что дает возможность указывать какие именно товары находятся в заказе.

Получите список из пяти клиентов, которые потратили больше всего денег в интернет-магазине.

Учитывайте только выполненные заказы.
Выведите четыре поля: id, фамилию и имя клиента, а также потраченную им сумму (в поле value).

Данные отсортируйте по сумме в обратном порядке.

**Решение:**

```MySQL
SELECT users.id AS id, users.last_name, users.first_name,SUM(products.price) AS value
FROM orders
JOIN users ON users.id=orders.user_id
JOIN orders_details ON orders_details.order_id=orders.id
JOIN products ON products.id=orders_details.product_id
WHERE orders.status = 'success'
GROUP BY orders.id
ORDER BY value DESC
LIMIT 5;
```

Продолжаем работать с базой данных интернет-магазина:

1. Таблица orders содержит список заказов пользователей и связана с users отношением один ко многим через поле user_id.

2. Также orders связана отношением многие ко многим с таблицей products через таблицу orders_details, что дает возможность указывать какие именно товары находятся в заказе.

Заказы со статусом «new» считаются корзиной. Измените состав товаров в корзине Александра Дмитриева.
    1. Добавьте в корзину холодильник.
    2. Удалите из корзины пылесос.

**Решение:**

```MySQL
INSERT INTO orders_details
SET order_id = 13, product_id = 2;
      
DELETE od FROM orders_details as od 
       JOIN orders as o ON o.id = od.order_id
       JOIN products as p ON p.id = od.product_id
       JOIN users as u ON u.id = o.user_id
WHERE p.name = 'Пылесос' AND u.id = 10;
```

Таблицы ниже содержат информацию о наличии обуви разных моделей, цветов и размеров в магазинах города.

В один из магазинов пришла девушка и решила примерить розовые Туфли Elite (EN1345) 39 размера, однако в этом магазине её размера не было и продавщица подошла к компьютеру, чтобы посмотреть в каких магазинах данная модель есть.

Напишите SQL запрос, который вернет все магазины, в которых есть нужная обувь. В финальной таблице должно быть два столбца: название магазина (shop) и количество пар обуви в нём (quantity).

Данные нужно отсортировать по количеству пар обуви.

**Решение:**

```MySQL
SELECT shops.name AS shop, stock.quantity AS quantity
FROM shops
JOIN stock ON shops.id = stock.shop_id
WHERE stock.color_id = 7
AND stock.size = 39
AND stock.model_id = 8;
```